In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def recall_at_k(query_vector, X, relevant_indices, k=5):
    """
    Calcula el recall@k dado un vector de consulta y una matriz TF-IDF.
    """
    similarities = cosine_similarity(query_vector, X).flatten()
    top_k_indices = np.argsort(similarities)[::-1][:k]
    hits = len(set(top_k_indices) & set(relevant_indices))
    recall = hits / len(relevant_indices) if relevant_indices else 0
    return recall

In [ ]:
def average_context_size(chunks):
    """
    Calcula el tamaño medio (en palabras) de los chunks recuperados.
    """
    sizes = [len(c.split()) for c in chunks]
    return np.mean(sizes)


In [ ]:
def evaluate_model(model, queries, ground_truth):
    """
    Evalúa el modelo RAG con varias consultas.
    - queries: lista de textos de consulta.
    - ground_truth: lista de índices relevantes por consulta.
    """
    recalls, context_sizes = [], []

    for i, query in enumerate(queries):
        print(f"\n🔎 Evaluando consulta {i+1}: {query}")
        results = model.query(query, top_k=5)
        retrieved_chunks = [r[0] for r in results]
        recall = recall_at_k(
            model.vectorizer.transform([query]),
            model.X,
            ground_truth[i],
            k=5,
        )
        recalls.append(recall)
        context_sizes.append(average_context_size(retrieved_chunks))

    avg_recall = np.mean(recalls)
    avg_context = np.mean(context_sizes)

    print("\n📈 Resultados promedio:")
    print(f"Recall@5 promedio: {avg_recall:.3f}")
    print(f"Tamaño medio de contexto: {avg_context:.1f} palabras")

    return avg_recall, avg_context